# 1. Library and Data Loading

## 1.1 Load the required Libraries

In [ ]:
import torch, torchvision
import torchvision.transforms as transforms
import json
import os
import statistics
from fastai.learner import Learner
import torch.nn.functional as F
from fastai.metrics import accuracy, Precision, Recall
import fastai.callback.schedule
from fastai.optimizer import OptimWrapper
from torch import optim
from functools import partial
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
from torch import nn
from fastai.data.core import DataLoaders
from fastai.data.all import *
from fastai.vision.all import *
from sklearn.model_selection import train_test_split

## 1.2 Data Loading

Create helper functions to automatically generate dataloaders for both the training and test datasets.

In [ ]:
def create_training_dataloaders(dataset):
    '''
    A helper function that automatically creates a dataloader for a given dataset
    where the dataset is the training data.
    Automatically splits the dataloader into a training and validation dataset, with
    the split being 80:20 training:validation.
    '''

    filename = repr(dataset).split()[1].strip("'").strip("2024_A2_datasets/")
    if filename == 'CIFAR10.npz':
        X_train = normalise_data(torch.tensor(dataset['X_tr'], dtype=torch.float32).permute(0, 3, 1, 2))
    else:
        X_train = normalise_data(torch.tensor(dataset['X_tr'], dtype=torch.float32).unsqueeze(1))
    Y_train = torch.tensor(dataset['S_tr'])

    # Create a datablock to handle the training information, with the validation set being
    # a randomly selected 20% of the training data
    train_data_block = DataBlock(
        blocks=(TransformBlock, CategoryBlock),
        get_x=lambda i: X_train[i],
        get_y=lambda i: Y_train[i].item(),
        splitter=RandomSplitter(valid_pct=0.2)
    )

    # Create the training dataloaders for the information
    train_dataloader = train_data_block.dataloaders(range(len(X_train)))
    train_dataloader.to('cuda')
    return train_dataloader

In [ ]:
def create_test_dataloader(dataset):
    '''
    A helper function that automatically creates a dataloader for a given
    test dataset.
    '''
    filename = repr(dataset).split()[1].strip("'").strip("2024_A2_datasets/")
    if filename == 'CIFAR10.npz':
        test_dataset = TensorDataset(normalise_data(torch.tensor(dataset['X_ts'], dtype=torch.float32).permute(0, 3, 1, 2)), torch.tensor(dataset['Y_ts']))
    else:
        test_dataset = TensorDataset(normalise_data(torch.tensor(dataset['X_ts'], dtype=torch.float32).unsqueeze(1)), torch.tensor(dataset['Y_ts']))

    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    test_dataloader.to('cuda')
    return test_dataloader

In [ ]:
def create_train_valid_split(dataset):
    '''
    Helper function to create a train/valid split for the training data.
    '''
    filename = repr(dataset).split()[1].strip("'").strip("2024_A2_datasets/")
    if filename == 'CIFAR10.npz':
        X_train = normalise_data(torch.tensor(dataset['X_tr'], dtype=torch.float32).permute(0, 3, 1, 2))
    else:
        X_train = normalise_data(torch.tensor(dataset['X_tr'], dtype=torch.float32).unsqueeze(1))

    Y_train = torch.tensor(dataset['S_tr'])

    return train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

# 2. Preprocessing

A helper preprocessing function to normalise data to the range of 0-1. This function is used in the data loading functions to normalise the data provided to the dataloaders.

In [ ]:
def normalise_data(tensor) :
    return torch.div(tensor, 255.0)

# 3. Custom Model and Loss Definitions

## 3.1 CNN Model Definition

In [ ]:
class Net(nn.Sequential):
    def __init__(self, input_dimension_size, conv_layers, conv_dimensions, fc_layers, fc_neurons, rgb):
        # Setup related stuff
        layers = []
        output_dim = input_dimension_size - conv_layers*conv_dimensions + conv_layers
        rgb_channels = 3 if rgb else 1

        for i in range(conv_layers):
            layers.append(nn.Conv2d(rgb_channels, rgb_channels, conv_dimensions))
            layers.append(nn.LeakyReLU())

        layers.append(nn.Flatten())

        if fc_layers > 1:
            layers.append(nn.Linear(rgb_channels * output_dim**2, fc_neurons))
            layers.append(nn.LeakyReLU())
        else:
            layers.append(nn.Linear(rgb_channels * output_dim**2, 4))

        if fc_layers > 2:
            for i in range(fc_layers - 2):
                layers.append(nn.Linear(fc_neurons, fc_neurons))
                layers.append(nn.LeakyReLU())

        if fc_layers >= 2:
            layers.append(nn.Linear(fc_neurons, 4))

        super().__init__(*layers)

## 3.2 Custom Loss Definition

In [ ]:
class NoisyCrossEntropyLoss(nn.Module):
    def __init__(self, transition_matrix=None):
        super(NoisyCrossEntropyLoss, self).__init__()
        self.register_buffer("transition_matrix", transition_matrix)

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        eps = torch.finfo().smallest_normal
        prob = F.softmax(x, dim=1)
        if self.transition_matrix is not None:
            prob = torch.matmul(prob, self.transition_matrix)
        loss = F.nll_loss(torch.log(prob + eps), y)
        return loss

# 4. Hyperparameter Exploration

## 4.1 Helper Functions

### 4.1.1 Function to find a transition matrix.

In [ ]:
def find_transition_matrix(model, train_dataloader, num_classes):
    '''
    Finds the transition matrix for a trained model.
    model: The trained model used to estimate the transition matrix.
    train_dataloader: The pytorch dataloader object that contains all the training information.
    num_classes: The number of unique classes/labels in the dataset.
    '''

    # Set up the model to be in evaluation mode
    model.eval()

    # Create the empty prediction results object, with enough columns
    # for each class
    predictions = torch.empty(0, num_classes)

    # Create the transition matrix object
    transition_matrix = np.zeros((num_classes, num_classes))

    # Go through each training batch
    for data, targets in train_dataloader:
        # Make predictions without calculating gradients, since
        # we are only interested in the outputs instead of training
        with torch.inference_mode():
            outputs = model(data).cpu()
            output = F.softmax(outputs, dim=1)
            predictions = torch.cat((predictions, output))
    # For each class, find the anchor point and set the column of data to that anchor point's probabilities
    for i in range(num_classes):
        transition_matrix[:, i] = predictions[torch.argmax(predictions[:, i]), :]

    # Return the model to training mode
    model.train()

    transition_matrix = torch.from_numpy(transition_matrix).to(torch.float32)
    return transition_matrix

### 4.1.2 Create a training wrapper to isolate the training functionality.

In [ ]:
def create_custom_dataloaders(X_train, Y_train, X_valid, Y_valid):
    '''
    A helper function that automatically creates dataloaders.
    '''
    # Create the datasets for training and validation
    train_dataset = TensorDataset(X_train, Y_train)
    valid_dataset = TensorDataset(X_valid, Y_valid)

    # Create the training and validation dataloaders for the information
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

    # Move the dataloaders to the GPU
    train_dataloader.to('cuda')
    valid_dataloader.to('cuda')

    return train_dataloader, valid_dataloader

In [ ]:
def custom_train(model, training_dataloaders, num_epochs=10, transition_matrix=None):
    '''
    Function that trains a model.
    model: The model being trained.
    training_dataloaders: The training and validation dataloaders.
    num_epochs: The number of epochs to train this model for.
    transition_matrix: The transition matrix to use during training. Note this is optional.
    '''
    # Create the required optimiser and learner objects, using an Adam optimiser,
    # a cross-entropy loss, and validation accuracy as the main metric.
    opt_func = partial(OptimWrapper, opt=optim.Adam)

    # Create the learner object that contains all this information
    loss_fn = NoisyCrossEntropyLoss(transition_matrix) if transition_matrix != None else NoisyCrossEntropyLoss()
    learn = Learner(
        training_dataloaders,
        model,
        loss_func=loss_fn,
        opt_func=opt_func,
        metrics=accuracy
    )

    # Make sure we get training metrics too
    learn.recorder.train_metrics = True

    # Train the model
    learn.fit(n_epoch=num_epochs)

    # Return the model and the recorder
    return learn.model, learn.recorder

### 4.1.3 Create a testing wrapper to isolate the testing functionality.

In [ ]:
def custom_test(dataset, model):
    # Set the model to evaluation mode
    model.eval()

    # Test the model
    counter = 0
    correct = 0.0
    # For each test example, count whether the model produces the correct class
    with torch.inference_mode():
        for test in create_test_dataloader(dataset):
            output = model(test[0])
            predicted_class = torch.argmax(output)
            if predicted_class == test[1].item():
                correct += 1
            counter += 1
    test_accuracy = correct/counter

    # Bring the model back to training mode
    model.train()

    return test_accuracy

### 4.1.4 Set up each of the hyperparameters.

In [ ]:
conv_layers = [1, 2, 5, 10]
conv_dimension_size = [2, 4, 6, 8, 10]
fc_layers = [1, 2, 5, 10]
fc_neurons = [100, 500, 1000]
num_epochs = [10, 100]

### 4.1.5 Create a custom grid search function to undertake grid search.

In [ ]:
from IPython.display import clear_output
def custom_grid_search(dataset, **hyperparameters):

    # Set up some variables to manage the grid search
    counter = 0
    total_searches = len(hyperparameters['conv_layers']) * len(hyperparameters['conv_dimension_size']) * len(hyperparameters['fc_layers']) * len(hyperparameters['fc_neurons']) * len(hyperparameters['num_epochs'])
    best_model = {
        'trial': None,
        'valid': False,
        'num_epochs': None,
        'conv_layers': None,
        'conv_dimension_size': None,
        'fc_layers': None,
        'fc_neurons': None,
        'avg_val_accuracy': 0,
        'transition_matrix': None
    }

    match dataset:
        case 'CIFAR10':
            data = np.load('2024_A2_datasets/CIFAR10.npz')
            input_dim_size = 32
            rgb = True
            transition_matrix = hyperparameters['find_transition_matrix']
        case 'FashionMINIST0.3':
            data = np.load('2024_A2_datasets/FashionMNIST0.3.npz')
            input_dim_size = 28
            rgb = False
            transition_matrix = torch.tensor([
                [0.7, 0.3, 0, 0],
                [0, 0.7, 0.3, 0],
                [0, 0, 0.7, 0.3],
                [0.3, 0, 0, 0.7]
            ])
        case 'FashionMINIST0.6':
            data = np.load('2024_A2_datasets/FashionMNIST0.6.npz')
            input_dim_size = 28
            rgb = False
            transition_matrix = torch.tensor([
                [0.4, 0.2, 0.2, 0.2],
                [0.2, 0.4, 0.2, 0.2],
                [0.2, 0.2, 0.4, 0.2],
                [0.2, 0.2, 0.2, 0.4]
            ])

    # Conduct the grid search
    for epochs in hyperparameters['num_epochs']:
        for num_conv_layers in hyperparameters['conv_layers']:
            for dimension_size in hyperparameters['conv_dimension_size']:
                for num_fc_layers in hyperparameters['fc_layers']:
                    for num_fc_neurons in hyperparameters['fc_neurons']:
                        counter += 1

                        # Check if this model has already been trained
                        if os.path.exists(f'{dataset}_cnn_results/trial_{counter}'):
                            continue

                        # Check if this combination of hyperparameters will be valid
                        if input_dim_size - num_conv_layers*dimension_size + num_conv_layers < 1:
                            with open(f'{dataset}_cnn_results/trial_{counter}', 'w') as f:
                                f.write(json.dumps({
                                    'trial': counter,
                                    'valid': False,
                                    'num_epochs': epochs,
                                    'conv_layers': num_conv_layers,
                                    'conv_dimension_size': dimension_size,
                                    'fc_layers': num_fc_layers,
                                    'fc_neurons': num_fc_neurons,
                                }))
                                continue

                        X_train, X_valid, Y_train, Y_valid = create_train_valid_split(data)

                        # Print some information for the user
                        print(f'Running model {counter} out of {total_searches} ({float(counter)/total_searches * 100:.2f}%)')
                        print(f'''Current model:
                        num_epochs: {epochs}
                        conv_layers: {num_conv_layers}
                        conv_dimension_size: {dimension_size}
                        fc_layers: {num_fc_layers}
                        fc_neurons: {num_fc_neurons}\n
                        ''')
                        print(f'''Best model: {best_model['avg_val_accuracy'] * 100:.2f}%
                        num_epochs: {best_model['num_epochs']}
                        conv_layers: {best_model['conv_layers']}
                        conv_dimension_size: {best_model['conv_dimension_size']}
                        fc_layers: {best_model['fc_layers']}
                        fc_neurons: {best_model['fc_neurons']}\n
                        ''')

                        # Define the model to train
                        model = Net(input_dim_size, num_conv_layers, dimension_size, num_fc_layers, num_fc_neurons, rgb)

                        # Create the training/validation dataloaders
                        dls = create_custom_dataloaders(X_train, Y_train, X_valid, Y_valid)
                        train_dataloaders = DataLoaders(dls[0], dls[1])

                        # Find the transition matrix of the data (if needed), otherwise just train the model
                        # using its existin transition matrix
                        if hyperparameters['find_transition_matrix']:
                            model, recorder = custom_train(model, train_dataloaders, num_epochs=10, transition_matrix=None)
                            transition_matrix = find_transition_matrix(model, train_dataloaders.train, 4)

                            # Once the transition matrix is found, create a new model that uses the transition matrix
                            model = Net(input_dim_size, num_conv_layers, dimension_size, num_fc_layers, num_fc_neurons, rgb)
                            model.cuda()
                            model, recorder = custom_train(model, train_dataloaders, num_epochs=epochs, transition_matrix=transition_matrix)
                        else:
                            model, recorder = custom_train(model, train_dataloaders, num_epochs=epochs, transition_matrix=transition_matrix)

                        print(model)

                        clear_output()

                        # Set up the model's result information
                        model_info = {
                            'trial': counter,
                            'valid': True,
                            'num_epochs': epochs,
                            'conv_layers': num_conv_layers,
                            'conv_dimension_size': dimension_size,
                            'fc_layers': num_fc_layers,
                            'fc_neurons': num_fc_neurons,
                            'avg_val_accuracy': recorder.values[0][-1],
                            'transition_matrix': transition_matrix.tolist()
                        }

                        # Save the trial information to the hard disk
                        with open(f'{dataset}_cnn_results/trial_{counter}', 'w') as f:
                            f.write(json.dumps(model_info))

                        # Replace the best model with the new best model
                        if model_info['avg_val_accuracy'] > best_model['avg_val_accuracy']:
                            best_model = model_info

## 4.2 Dataset 1: FashionMINIST0.3.npz

In [ ]:
custom_grid_search('FashionMINIST0.3', find_transition_matrix=False, conv_layers=conv_layers, conv_dimension_size=conv_dimension_size, fc_layers=fc_layers, fc_neurons=fc_neurons, num_epochs=num_epochs)

## 4.3 Dataset 2: FashionMINIST0.6.npz

In [ ]:
custom_grid_search('FashionMINIST0.6', find_transition_matrix=False, conv_layers=conv_layers, conv_dimension_size=conv_dimension_size, fc_layers=fc_layers, fc_neurons=fc_neurons, num_epochs=num_epochs)

## 4.4 Dataset 3: CIFAR.npz

In [ ]:
custom_grid_search('CIFAR10', find_transition_matrix=True, conv_layers=conv_layers, conv_dimension_size=conv_dimension_size, fc_layers=fc_layers, fc_neurons=fc_neurons, num_epochs=num_epochs)

# 5. Results

## 5.1 Find the best performing model for each dataset (judged by highest validation accuracy)

In [ ]:
def get_best_model(dataset):
    path = f'{dataset}_cnn_results'

    best_model = None
    best_accuracy = 0
    for trial in os.listdir(path):
        with open(f'{path}/{trial}', 'r') as f:
            results = json.loads(f.read())
            if results['valid'] and results['avg_val_accuracy'] > best_accuracy:
                best_model = results
                best_accuracy = results['avg_val_accuracy']

    return best_model

In [ ]:
best_cifar_cnn = get_best_model('CIFAR10')
best_fashion3_cnn = get_best_model('FashionMINIST0.3')
best_fashion6_cnn = get_best_model('FashionMINIST0.6')

print(best_cifar_cnn)
print(best_fashion3_cnn)
print(best_fashion6_cnn)

{'trial': 282, 'valid': True, 'num_epochs': 100, 'conv_layers': 1, 'conv_dimension_size': 8, 'fc_layers': 2, 'fc_neurons': 1000, 'avg_val_accuracy': 0.5945000052452087, 'transition_matrix': [[0.9996451139450073, 9.148226620681044e-11, 3.619066410465166e-05, 5.073326065030415e-06], [0.0002931593044195324, 0.9999998807907104, 5.171649536350742e-05, 2.4422119437872425e-08], [5.638430957333185e-05, 9.323639460490085e-08, 0.999903678894043, 5.463606385092135e-07], [5.37204277861747e-06, 5.1986872584564026e-09, 8.429781701124739e-06, 0.9999943971633911]]}
{'trial': 33, 'valid': True, 'num_epochs': 10, 'conv_layers': 1, 'conv_dimension_size': 6, 'fc_layers': 5, 'fc_neurons': 1000, 'avg_val_accuracy': 0.6637499928474426, 'transition_matrix': [[0.699999988079071, 0.30000001192092896, 0.0, 0.0], [0.0, 0.699999988079071, 0.30000001192092896, 0.0], [0.0, 0.0, 0.699999988079071, 0.30000001192092896], [0.30000001192092896, 0.0, 0.0, 0.699999988079071]]}
{'trial': 42, 'valid': True, 'num_epochs': 10,

## 5.2 Evaluate model performance on test set
For each CNN model, train it on a random 80:20 split of training:validation data, find the transition matrix if necessary, and then obtain the test accuracy. Repeat this 10 times.

In [ ]:
def train_transition_test_loop(dataset, best_model, find_the_transition_matrix=True):

    # Set up variables for the relevant dataset
    match dataset:
        case 'CIFAR10':
            data = np.load('2024_A2_datasets/CIFAR10.npz')
            input_dim_size = 32
            rgb = True
            transition_matrix = None
            find_the_transition_matrix = find_the_transition_matrix
        case 'FashionMNIST0.3':
            data = np.load('2024_A2_datasets/FashionMNIST0.3.npz')
            input_dim_size = 28
            rgb = False
            transition_matrix = None
            #DEBUG - uncomment for submission
    # torch.tensor([
            #     [0.7, 0.3, 0, 0],
            #     [0, 0.7, 0.3, 0],
            #     [0, 0, 0.7, 0.3],
            #     [0.3, 0, 0, 0.7]
            # ])
        case 'FashionMNIST0.6':
            data = np.load('2024_A2_datasets/FashionMNIST0.6.npz')
            input_dim_size = 28
            rgb = False
            transition_matrix = None
            #DEBUG - uncomment for submission
            # torch.tensor([
            #     [0.4, 0.2, 0.2, 0.2],
            #     [0.2, 0.4, 0.2, 0.2],
            #     [0.2, 0.2, 0.4, 0.2],
            #     [0.2, 0.2, 0.2, 0.4]
            # ])



    # Conduct 10 experiments on the datasets
    test_accuracies = []
    for i in range(10):
        # Create the training dataloaders (random split of 80:20 training:validation)
        train_valid_dataloaders = create_training_dataloaders(data)

        # Create the model
        model = Net(
            input_dim_size,
            best_model['conv_layers'],
            best_model['conv_dimension_size'],
            best_model['fc_layers'],
            best_model['fc_neurons'],
            rgb
        )

        # Move the model to the GPU
        model.cuda()

        # Find the transition matrix of the data (if needed), otherwise just train the model
        # using its existing transition matrix
        if find_the_transition_matrix:
            model, recorder = custom_train(model, train_valid_dataloaders, num_epochs=10, transition_matrix=None)
            transition_matrix = find_transition_matrix(model, train_valid_dataloaders.train, 4)

            # Once the transition matrix is found, create a new model that uses the transition matrix
            model = Net(
                input_dim_size,
                best_model['conv_layers'],
                best_model['conv_dimension_size'],
                best_model['fc_layers'],
                best_model['fc_neurons'],
                rgb
            )
            model.cuda()
            model, recorder = custom_train(model, train_valid_dataloaders, num_epochs=best_model['num_epochs'], transition_matrix=transition_matrix)
        else:
            model, recorder = custom_train(model, train_valid_dataloaders, num_epochs=best_model['num_epochs'], transition_matrix=transition_matrix)

        # Test the model on the test dataset
        test_accuracy = custom_test(data, model)
        test_accuracies.append(test_accuracy)

    if transition_matrix:
        return test_accuracies, statistics.mean(test_accuracies), statistics.pstdev(test_accuracies), transition_matrix.tolist()

    return test_accuracies, statistics.mean(test_accuracies), statistics.pstdev(test_accuracies), transition_matrix

In [ ]:
if not os.path.exists('cnn_cifar_stats'):
    cifar_stats = train_transition_test_loop('CIFAR10', best_cifar_cnn, find_the_transition_matrix=True)
    with open('cnn_cifar_stats', 'w') as f:
        f.write(str(cifar_stats))
if not os.path.exists('cnn_fashion3_stats'):
    fashion3_stats = train_transition_test_loop('FashionMNIST0.3', best_fashion3_cnn, find_the_transition_matrix=False)
    with open('cnn_fashion3_stats', 'w') as f:
        f.write(str(fashion3_stats))
if not os.path.exists('cnn_fashion6_stats'):
    fashion6_stats = train_transition_test_loop('FashionMNIST0.6', best_fashion6_cnn, find_the_transition_matrix=False)
    with open('cnn_fashion6_stats', 'w') as f:
        f.write(str(fashion6_stats))

# Train the two fashion datasets again to see what kind of transition matricies they produce, evaluating the effectiveness
# of creating transition matricies
if not os.path.exists('cnn_fashion3_stats_tm'):
    fashion3_stats_tm = train_transition_test_loop('FashionMNIST0.3', best_fashion3_cnn, find_the_transition_matrix=True)
    with open('cnn_fashion3_stats_tm', 'w') as f:
        f.write(str(fashion3_stats_tm))
if not os.path.exists('cnn_fashion6_stats_tm'):
    fashion6_stats_tm = train_transition_test_loop('FashionMNIST0.6', best_fashion6_cnn, find_the_transition_matrix=True)
    with open('cnn_fashion6_stats_tm', 'w') as f:
        f.write(str(fashion6_stats_tm))

if not os.path.exists('cnn_cifar_without_tm'):
    cifar_without_tm = train_transition_test_loop('CIFAR10', best_cifar_cnn, find_the_transition_matrix=False)
    with open('cnn_cifar_without_tm', 'w') as f:
        f.write(str(cifar_without_tm))

if not os.path.exists('cnn_fashion3_without_tm'):
    fashion3_without_tm = train_transition_test_loop('FashionMNIST0.3', best_fashion3_cnn, find_the_transition_matrix=False)
    with open('cnn_fashion3_without_tm', 'w') as f:
        f.write(str(fashion3_without_tm))

if not os.path.exists('cnn_fashion6_without_tm'):
    fashion6_without_tm = train_transition_test_loop('FashionMNIST0.6', best_fashion6_cnn, find_the_transition_matrix=False)
    with open('cnn_fashion6_without_tm', 'w') as f:
        f.write(str(fashion6_without_tm))

print(f'CIFAR stats: {cifar_stats}\n')
print(f'Fashion3 stats: {fashion3_stats}\n')
print(f'Fashion6 stats: {fashion6_stats}\n')
print()
print(f'Fashion3 predicted transition matrix: {fashion3_stats_tm}\n')
print(f'Fashion6 predicted transition matrix: {fashion6_stats_tm}\n')

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.134896,0.471250,1.101318,0.541000,00:01
1,1.021765,0.575500,1.077713,0.558750,00:01
2,0.967927,0.599312,1.070238,0.571750,00:01
3,0.939797,0.617625,1.025563,0.588500,00:01
4,0.866223,0.648188,1.023313,0.603000,00:01
5,0.839341,0.669625,1.034716,0.601250,00:01
6,0.743126,0.706000,1.074480,0.585250,00:01
7,0.677847,0.735062,1.145095,0.585750,00:01
8,0.622993,0.772375,1.255060,0.566250,00:01
9,0.495768,0.814687,1.336233,0.560500,00:02


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.109755,0.493750,1.105173,0.536250,00:02
1,1.001264,0.569687,1.067709,0.559000,00:03
2,0.984496,0.594375,1.064471,0.560250,00:03
3,0.935357,0.622500,1.037245,0.582250,00:03
4,0.890430,0.646250,1.013358,0.594750,00:03
5,0.850891,0.667250,1.020254,0.591500,00:03
6,0.792324,0.690562,1.074054,0.591500,00:03
7,0.738545,0.717000,1.096299,0.577250,00:03
8,0.652567,0.754062,1.195201,0.570500,00:03
9,0.576711,0.792188,1.179139,0.561750,00:03


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.136005,0.473250,1.057651,0.570000,00:02
1,1.051490,0.554187,1.076671,0.544500,00:03
2,1.017357,0.587563,1.012456,0.585250,00:03
3,0.956152,0.614563,1.036631,0.578250,00:03
4,0.892383,0.642187,1.015641,0.592500,00:03
5,0.812251,0.676750,1.043516,0.606000,00:03
6,0.745752,0.709750,1.093874,0.580500,00:03
7,0.647429,0.753688,1.130713,0.602500,00:03
8,0.551017,0.799125,1.221132,0.578750,00:03
9,0.440452,0.849187,1.333732,0.558500,00:03


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.093088,0.486000,1.086229,0.552500,00:03
1,1.029601,0.572625,1.143577,0.528500,00:03
2,0.963120,0.608250,1.047743,0.564500,00:03
3,0.908414,0.632875,1.068709,0.576000,00:03
4,0.857102,0.659312,1.056841,0.587000,00:03
5,0.794573,0.685375,1.106405,0.567250,00:03
6,0.717366,0.718250,1.119033,0.570000,00:03
7,0.645270,0.757875,1.217345,0.563000,00:03
8,0.552848,0.801188,1.250565,0.562000,00:03
9,0.438640,0.843188,1.389776,0.570500,00:03


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.232854,0.381125,1.287032,0.443000,00:03
1,1.103559,0.504250,1.059276,0.549250,00:03
2,1.025049,0.567187,1.057984,0.557000,00:03
3,0.987782,0.599500,0.998341,0.590750,00:03
4,0.945059,0.617625,1.003583,0.592000,00:03
5,0.942801,0.628062,1.004113,0.587250,00:03
6,0.891145,0.642687,0.955328,0.621750,00:03
7,0.847375,0.662250,0.962974,0.617500,00:03
8,0.849970,0.671188,0.993497,0.602500,00:03
9,0.791995,0.685750,1.003776,0.615250,00:03


...

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.358978,0.365312,1.350068,0.375833,00:03
1,1.346339,0.379323,1.350517,0.376042,00:03
2,1.334155,0.385938,1.352895,0.372708,00:03
3,1.330974,0.389375,1.357358,0.366458,00:03
4,1.316777,0.396042,1.367364,0.363125,00:03
5,1.299196,0.406875,1.376930,0.358333,00:03
6,1.277965,0.425833,1.396300,0.352500,00:03
7,1.243017,0.448438,1.418160,0.341875,00:03
8,1.198549,0.479427,1.455784,0.330000,00:03
9,1.142542,0.515885,1.503403,0.327708,00:03


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.355240,0.358021,1.349386,0.372917,00:03
1,1.350443,0.376771,1.347989,0.382708,00:03
2,1.342059,0.382292,1.353544,0.377083,00:03
3,1.331009,0.386927,1.354544,0.372500,00:03
4,1.320342,0.394844,1.362773,0.370625,00:03
5,1.309088,0.403490,1.366889,0.366458,00:03
6,1.289401,0.420417,1.374058,0.358125,00:03
7,1.256350,0.437031,1.406125,0.351875,00:03
8,1.225742,0.457865,1.423898,0.329167,00:03
9,1.173126,0.485521,1.451681,0.339583,00:03


epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.352259,0.361354,1.349916,0.383333,00:03
1,1.350355,0.379115,1.347161,0.380208,00:03
2,1.343200,0.383802,1.349179,0.382500,00:03
3,1.340418,0.389115,1.350580,0.376875,00:03
4,1.327478,0.393958,1.355215,0.375833,00:03
5,1.312054,0.404219,1.363052,0.377917,00:03
6,1.290305,0.417760,1.386116,0.357500,00:03
7,1.264244,0.437969,1.402189,0.339375,00:03
8,1.217914,0.465156,1.436907,0.345833,00:03
9,1.175491,0.493490,1.468287,0.335833,00:03


CIFAR stats: ([0.491, 0.482, 0.447], [0.247, 0.243, 0.261])



NameError: name 'fashion3_stats' is not defined